<a href="https://colab.research.google.com/github/CarlosBurgosB/Projects/blob/main/Greedy_Algorithms_Applications_in_Videogame_Itemization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Greedy Algorithms Applications in Videogame Itemization

The following example, for an specific case in League of Legends, shows how itemization in videogames can be optimized using a greedy algorithm, further discussion in section **Expanding the model**.

This model aims to obtain the best 3 item combination to tank damage coming from an AD Carry. 

A few assumptions were made for the model:


*   The incoming damage from the AD Carry was estimated from a lvl 13 Aphelios building Berserker's Greaves, Immortal Shieldbow and Lord Dominik's regards. Since there's a crit chance involved, the damage is the expected value assuming 40% of the AA crits = 277
*  Analytically, the first item for the tank in the model will always be Plated Stellcaps, since all tanks need to build boots and PS are de facto the best boots against Basic Attack damage. 
*   Anathema's chain was not included in the model, if it was the output would always have been PS + Mythic + Anathema's, but building the item and using the active on an AD Carry gaves up a lot in solo lane.
*   The model takes in account the lifeline effect from Sterak's Cage, Randuin's active, passive of the items and effects of Cut Down and Lord Dominik's Regards.
*   The model takes in account the base HP and Armor of a tank at lvl 13.

Execute all cells and go to the **Output** section to see the resulting item combination.

## Code


In [ ]:
#Data of items entered manually (didn't have time to learn to export item data from riot api)
#The notation for the elements of each list is [Item Name, effective HP, Armor]

#Mythic items
Mythic = [["Sunfire Aegis",350+50,35], ["Frostfire Gauntlet",350+100,25], ["Turbo Chemtank",350+50 ,25 ]]

#Legendary items, excluding edge cases (sterak's and randuin)
Legendary = [["Chempunk Chainsword",250,0],["Dead Man's Plate",300,45],["Knight's Vow",400,0],["Redemption",200,0],["Warmog's armor",800,0],
            ["Demonic Embrace",450,0],["Titanic Hydra",500,0],["Thornmail",350,60],["Zeke's Convergence",250,25]]

#Sterak's
steraks = ["Sterak's Gage",400,0]
#Randuin's
randuin = ["Randuin's Omen",250,80]
#Frozen Heart
frozen = ["Frozen Heart",0,80]
#Plated Steelcaps Armor
PS_arm = 20

In [ ]:
#Greedy algorithm for non edge cases
max_non_edge = 0
max_items_non_edge = [0]
for i in range(0,3):
  for j in range(0,9):
    HP = 1615 + Mythic[i][1] + Legendary[j][1]
    #effective armor as (base + items - lethality) * (1-armor pen percentage from LDR)
    EffArmor = (83.5 + PS_arm +Mythic[i][2]+Legendary[j][2]-4.5 )*0.65
    #damage amp from cut down and dominik (same formula for both)
    cd_dr_dmg_increase = ((HP-1494)/1494-0.1)*1/9+0.05
    #since aphelios has both it is applied multiplicatively
    #0.88 multiplier from plated steelcaps passive
    damage_per_second = 277* 1.63 *(1+cd_dr_dmg_increase)*(1+cd_dr_dmg_increase)*(100/(100+EffArmor)) * 0.88
    time = HP/damage_per_second
    if time > max_non_edge:
      max_non_edge = time
      max_items_non_edge[0] = ["Plated Steelcaps", Mythic[i][0], Legendary[j][0]]



In [ ]:
#Sterak's edge cases
max_st_edge = 0
max_items_st_edge = [0]
for i in range(0,3):
  HP = 1615 + Mythic[i][1] + steraks[1]
  #effective armor as (base + items - lethality) * (1-armor pen percentage from LDR)
  EffArmor = (83.5 + PS_arm +Mythic[i][2]+steraks[2]-4.5 )*0.65
  #damage amp from cut down and dominik (same formula for both) 
  cd_dr_dmg_increase = ((HP-1494)/1494-0.1)*1/9+0.05
  #since aphelios has both it is applied multiplicatively
  #0.88 multiplier from plated steelcaps passive
  damage_per_second = 277* 1.63 *(1+cd_dr_dmg_increase)*(1+cd_dr_dmg_increase)*(100/(100+EffArmor)) * 0.88
  shield = 100+0.4*HP
  if shield > damage_per_second*4:
    HP = HP + damage_per_second*4
  else:
    HP = HP + shield
  time = HP/damage_per_second
  if time > max_st_edge:
    max_st_edge = time
    max_items_st_edge[0] = ["Plated Steelcaps", Mythic[i][0], steraks[0]]


In [ ]:
#Randuin's edge cases
#similar to non-edge cases, however, a flat damage reduction is add to each AA because of Rock Solid passive
max_r_edge = 0
max_items_r_edge = [0]
for i in range(0,3):
  HP = 1615 + Mythic[i][1] + randuin[1]
  #Rock solid passive
  Damage_reduction = 5 + int(HP/1000)*3.5
  #effective armor as (base + items - lethality) * (1-armor pen percentage from LDR)
  EffArmor = (83.5 + PS_arm +Mythic[i][2]+randuin[2]-4.5 )*0.65
  #damage amp from cut down and dominik (same formula for both)
  cd_dr_dmg_increase = ((HP-1494)/1494-0.1)*1/9+0.05
  #since aphelios has both it is applied multiplicatively
  #0.88 multiplier from plated steelcaps passive
  damage_per_second = 277* 1.63 *(1+cd_dr_dmg_increase)*(1+cd_dr_dmg_increase)*(100/(100+EffArmor)) * 0.88 - Damage_reduction
  #AA Damage the first 4 seconds is reduced by randuin's active
  AA_r_active_dmg = 213*0.9*0.2 + 213*0.9*1.75*0.8
  #so the dps while randuin's is active is
  dps_active = AA_r_active_dmg * 1.63 *(1+cd_dr_dmg_increase)*(1+cd_dr_dmg_increase)*(100/(100+EffArmor)) * 0.88 - Damage_reduction
  time = HP/dps_active
  #if the tank lives more than those 4 seconds the time is 4 + time for remaining health
  if time>4:
    time = 4 + (HP-dps_active*4)/damage_per_second
  if time > max_r_edge:
    max_r_edge = time
    max_items_r_edge[0] = ["Plated Steelcaps", Mythic[i][0], randuin[0]]

In [ ]:
#Frozen Heart edge cases
#analog to randuin, without the active
#instead of randuin's active, the AS of the enemy is reduced
max_f_edge = 0
max_items_f_edge = [0]
for i in range(0,3):
  HP = 1615 + Mythic[i][1] + frozen[1]
  #Rock solid passive
  Damage_reduction = 5 + int(HP/1000)*3.5
  #effective armor as (base + items - lethality) * (1-armor pen percentage from LDR)
  EffArmor = (83.5 + PS_arm +Mythic[i][2]+frozen[2]-4.5 )*0.65
  #damage amp from cut down and dominik (same formula for both)
  cd_dr_dmg_increase = ((HP-1494)/1494-0.1)*1/9+0.05
  #since aphelios has both items it is applied multiplicatively
  #0.88 multiplier from plated steelcaps passive
  #attack speed is not 1.63
  a_speed = 1.63*0.8
  damage_per_second = 277* a_speed *(1+cd_dr_dmg_increase)*(1+cd_dr_dmg_increase)*(100/(100+EffArmor)) * 0.88 - Damage_reduction
  time = HP/damage_per_second
  if time > max_f_edge:
    max_f_edge = time
    max_items_f_edge[0] = ["Plated Steelcaps", Mythic[i][0], frozen[0]]

## Output

In [ ]:
#Output (best item combination)
maximo = max(max_non_edge,max_st_edge,max_r_edge,max_f_edge)
print("The best item combination is: ")
if maximo == max_non_edge:
  print(max_items_non_edge)
elif maximo == max_st_edge:
  print(max_items_st_edge)
elif maximo == max_r_edge:
  print(max_items_r_edge)
else:
  print(max_items_f_edge) 
print("With a time alive of " + str(maximo) + " seconds")

The best item combination is: 
[['Plated Steelcaps', 'Sunfire Aegis', 'Frozen Heart']]
With a time alive of 14.151774536026275 seconds


## Expanding the model

There are several ways of expanding the previous model:

*   Adding item slots, to optimize a full build.
*   Adding another damage source, for example a mage, optimizing survivability against both types of damage.
*   Adding true damage averaged per second to the time ecuation, to optimize the build against characters with true HP damage like Vayne.
*   Adding a restraint to get the best powerspike spending below a certain ammount of gold.

It's worth mentioning that this model can be used for Damage Carries, doing the inverse process, using the greedy algorithm to minimize the time the enemy is alive.

Concluding, algorithms to optimize itemization are heavily underused in esports, it seems coaching staffs and players rely in manually proving item combinations instead of hiring an item analyst with programming background to automate this task.




